### Uitwerkingen Bas Levering 500784954



In deze opgave classificeren afbeeldingen van bloemen uit de iris dataset met behulp van een neuraal netwerk.

In [216]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

Laad de dataset en maak een feature matrix X en een target vector y.

In [217]:
iris = sns.load_dataset('iris')

# we gebruiken .values om een Numpy array te krijgen in plaats van een Pandas DataFrame
X_iris = iris.drop('species', axis=1).values 
y_iris = iris['species'].values

In [218]:
y_iris = y_iris.reshape(-1, 1) # maak een array van array's, dit is nodig voor de volgende stap
print(y_iris[:10])

[['setosa']
 ['setosa']
 ['setosa']
 ['setosa']
 ['setosa']
 ['setosa']
 ['setosa']
 ['setosa']
 ['setosa']
 ['setosa']]


Een neuraal netwerk kan niet omgaan met categorische labels zoals de namen van de planten, maar heeft getallen als uitput nodig. We hebben hier drie klassen (setosa, versicolor, virginica) daarom maken we een target vector met per label drie waarden. Een 1 voor de eerste waarde correspondeert met 'setosa', een 1 voor de 2e waade met 'versicolor' en een 1 voor de derde waarde met 'virginica'.

Doe dit met behulp van het `OneHotEncoder` object uit sklearn.

In [219]:
enc = OneHotEncoder(handle_unknown='ignore')
y_enc = enc.fit_transform(y_iris).toarray()
enc_cat = enc.categories_

print(enc_cat)

[array(['setosa', 'versicolor', 'virginica'], dtype=object)]


We hebben de data nu in een geschikt formaat. Splits de data in training en test data.

In [220]:

X_train, X_test, y_train, y_test = train_test_split(X_iris, y_enc, test_size=0.2, random_state=6)


Maak een eenvoudig neuraal netwerk met 3 lagen, waarin elke neuron in een laag met elk neuron in de vorige laag verbonden is ('Dense'). 
Voeg voor de eeste laag een parameter `input_shape` toe die aangeeft hoeveel features een element in 'X' heeft.
Het aantal neuronen in de laaste laag is gelijk aan het aantal waarden in een label in `y`. 

In [221]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

model = Sequential()
model.add(layers.Dense(4, input_shape= (4,), activation = 'relu'))
model.add(layers.Dense(4, activation = 'relu'))
model.add(layers.Dense(3, activation = 'relu'))



Compileer en bekijk het netwerk door onderstaande code te runnen.

In [222]:
from tensorflow.keras.optimizers import Adam

model.compile(Adam(lr=0.01),'categorical_crossentropy',metrics=['accuracy'])

model.summary()

Model: "sequential_26"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_74 (Dense)             (None, 4)                 20        
_________________________________________________________________
dense_75 (Dense)             (None, 4)                 20        
_________________________________________________________________
dense_76 (Dense)             (None, 3)                 15        
Total params: 55
Trainable params: 55
Non-trainable params: 0
_________________________________________________________________


Nu kunnen we ons model trainen met behulp van de `fit` methode.

In [223]:
model.fit(X_train, y_train, epochs = 25, batch_size = 5)


Epoch 1/25
24/24 [==============================] - 0s 4ms/step - loss: 10.8797 - accuracy: 0.3250
Epoch 2/25
24/24 [==============================] - 0s 2ms/step - loss: 10.8797 - accuracy: 0.3250
Epoch 3/25
24/24 [==============================] - 0s 920us/step - loss: 10.8797 - accuracy: 0.3250
Epoch 4/25
24/24 [==============================] - 0s 920us/step - loss: 10.8797 - accuracy: 0.3250
Epoch 5/25
24/24 [==============================] - 0s 920us/step - loss: 10.8797 - accuracy: 0.3250
Epoch 6/25
24/24 [====================

Met `model_evaluate` kunnen we bepalen hoe goed het model werkt op de test data.

In [224]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(test_acc)

1/1 [==============================] - 0s 104ms/step - loss: 10.2081 - accuracy: 0.3667
0.36666667461395264


## Vragen
Experimenteer met bovenstaand model. Wat gebeurt er als je meer lagen toevoegt of een laag weghaalt of het aantal neuronen in een laag verandert? 

Probeer ook eens andere activatiefuncties dan `relu`, zoals `tanh` of `sigmoid`.

Welke invloed heeft het aantal epochs?
## Antwoorden:
1: De hoeveelheid Neuronen in een laag heeft invloed op hoe correct je model is, teveel of teweinig heeft invloed op het antwoord.
2: De Combinatie van relu tanh en sigmoid werkt het best
3: Epochs is het aantal rondes over de train set. dit kan het model beter maken maar ook overtrainen


We kunnen dezelfde aanpak gebruiken voor andere data, bijvoorbeeld voor het herkennen van cijfers. We gebruiken de `MNist` dataset die bestaat uit plaatjes van cijfers. Ieder plaatje bestaat uit 28 x 28 pixels.

In [225]:
from keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

import warnings
warnings.filterwarnings('ignore')

In [226]:
train_images.shape

(60000, 28, 28)

In [227]:
len(train_labels)

60000

In [228]:
print(train_labels)

[5 0 4 ... 5 6 8]


Een eenvoudig neuraal netwerk heeft een array van inputwaarden nodig i.p.v. een 2D afbeelding.
We maken van de inputs (waarden tussen 0 en 255) getallen tussen 0 en 1.

In [229]:
train_images = train_images.reshape((60000, 28 * 28)) # lijst van waarden i.p.v. 2D afbeelding
train_images = train_images.astype('float32') / 255 # getallen tussen 0 en 1

test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

Voor elk label (een cijfer) maken we een array met 10 waarden: 9 nullen en een één, waarbij de positie van de één aangeeft om welk cijfer het gaat (One hot encoding). Dit keer gebruiken we hiervoor de `keras` functie `to_categorical`.

In [230]:
from tensorflow.keras.utils import to_categorical

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

print(train_labels)

[[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]]


Maak nu een neuraal netwerk met twee lagen, een eerste laag met 512 neuronen en een output laag met 10 neuronen. Geef bij de eerste laag aan hoeveel inputs er zijn en wat de activatiefunctie is. De output laag heeft een `softmax` activatiefunctie.

In [231]:
model = Sequential()
model.add(layers.Dense(512, input_shape= (28*28,), activation = 'relu'))
model.add(layers.Dense(10, activation = 'softmax'))


compileer het netwerk.

In [232]:
model.compile(Adam(learning_rate=0.01),'categorical_crossentropy',metrics=['accuracy'])

model.summary()

Model: "sequential_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_77 (Dense)             (None, 512)               401920    
_________________________________________________________________
dense_78 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


train het netwerk 5 epochs met een batch_size van 128.

In [233]:
model.fit(train_images, train_labels, epochs=5, batch_size=128)


2021-09-15 16:28:40.655661: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 188160000 exceeds 10% of free system memory.


Epoch 1/5
469/469 [==============================] - 3s 6ms/step - loss: 0.2109 - accuracy: 0.9353

Evalueer het netwerk op de test set.

In [234]:

test_loss, test_acc = model.evaluate(test_images, test_labels)
print(test_acc)

 18/313 [>.............................] - ETA: 0s - loss: 0.0917 - accuracy: 0.9774 

2021-09-15 16:28:55.843585: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 31360000 exceeds 10% of free system memory.


313/313 [==============================] - 1s 2ms/step - loss: 0.1394 - accuracy: 0.9696

Experimenteer met verschillende netwerken. Varieer het aantal lagen, het aantal neuronen, de activatiefunties en het aantal epochs.

## Opmerkingen Bas Levering

Je kunt zien dat variatie in het aantal neuronen en epochs direct invloed heeft op de accuraatheid van ja train set.
